In [108]:
import pymysql
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window

In [109]:
def mysql_connect():
    return pymysql.connect(host='localhost', port=3306, user='user', password='pass123', db='uol')
sql = 'SELECT * FROM registration'

In [110]:
# Get pdf from pandas
pdf = pd.read_sql(sql, con=mysql_connect())

In [111]:
# Create data frame from pdf with spark
spark = SparkSession.builder.appName('singleUserService').getOrCreate()
df = spark.createDataFrame(pdf)
df.show()

+---------------+------------------+---------------------+--------------------+----------------------+-----------------------+-------------------------+----------------------------+-------------------+----------------------+---------------+--------------------------+--------------------------------+--------------------------------+------------------------------+---------------------+---------------------+--------------------------+------------------------+
|registration_id|registration_email|registration_password|   registration_name|registration_cellphone|registration_user_agent|registration_name_browser|registration_version_browser|registration_system|registration_gpu_model|registration_ip|registration_duration_time|registration_accepted_terms_time|registration_start_date_register|registration_end_date_register|registration_scroll_x|registration_scroll_y|registration_scroll_millis|registration_unique_hash|
+---------------+------------------+---------------------+--------------------

In [112]:
# count by ip
df2 = df.groupBy(
    'registration_ip'
).agg(
    F.count('*').alias('ip_count')
).withColumn(
    'ip_count_struct',
    F.struct('registration_ip', 'ip_count')
)

df2.show()

Traceback (most recent call last):
  File "/workspace/.pip-modules/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/workspace/.pip-modules/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 62, in worker
  File "/workspace/.pip-modules/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 525, in loads
    length = read_int(stream)
  File "/workspace/.pip-modules/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


+---------------+--------+------------------+
|registration_ip|ip_count|   ip_count_struct|
+---------------+--------+------------------+
|  187.95.19.132|       5|{187.95.19.132, 5}|
+---------------+--------+------------------+



In [113]:
# count by hash
df2 = df.groupBy(
    'registration_unique_hash'
).agg(
    F.count('*').alias('hash_count')
).withColumn(
    'hash_count_struct',
    F.struct('registration_unique_hash', 'hash_count')
)

df2.show()

+------------------------+----------+--------------------+
|registration_unique_hash|hash_count|   hash_count_struct|
+------------------------+----------+--------------------+
|    7c22d54fb53acd631...|         5|{7c22d54fb53acd63...|
+------------------------+----------+--------------------+



In [114]:
# count by scroll params
df2 = df.groupBy(
    'registration_scroll_y',
    'registration_scroll_millis'
).agg(
    F.count('*').alias('scroll_count')
).withColumn(
    'scroll_count_struct',
    F.struct('registration_scroll_y', 'scroll_count')
)

df2.show()

+---------------------+--------------------------+------------+--------------------+
|registration_scroll_y|registration_scroll_millis|scroll_count| scroll_count_struct|
+---------------------+--------------------------+------------+--------------------+
|      100|200|300|400|                     4|3|4|           1|{100|200|300|400, 1}|
| 100|200|300|400|5...|      4|4|2|4|2850|3|4|...|           1|{100|200|300|400|...|
| 100|200|300|400|5...|      3|4|4|4|4|4|2962|...|           1|{100|200|300|400|...|
| 100|200|300|400|5...|               3|3|3|3|2|3|           1|{100|200|300|400|...|
|  100|200|300|400|500|                   4|3|3|4|           1|{100|200|300|400|...|
+---------------------+--------------------------+------------+--------------------+



In [115]:
# count by register duration timestamp
df2 = df.groupBy(
    'registration_duration_time'
).agg(
    F.count('*').alias('duration_register_count')
).withColumn(
    'duration_count_struct',
    F.struct('registration_duration_time', 'duration_register_count')
)

df2.show()

+--------------------------+-----------------------+---------------------+
|registration_duration_time|duration_register_count|duration_count_struct|
+--------------------------+-----------------------+---------------------+
|                     14140|                      1|           {14140, 1}|
|                     14649|                      1|           {14649, 1}|
|                     14653|                      1|           {14653, 1}|
|                     10374|                      1|           {10374, 1}|
|                     13833|                      1|           {13833, 1}|
+--------------------------+-----------------------+---------------------+



Traceback (most recent call last):
  File "/workspace/.pip-modules/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/workspace/.pip-modules/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 62, in worker
  File "/workspace/.pip-modules/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 525, in loads
    length = read_int(stream)
  File "/workspace/.pip-modules/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
Traceback (most recent call last):
  File "/workspace/.pip-modules/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/workspace/.pip-modules/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 62, in worker
  File "/workspace/.pip-modules/lib/python3.8/site-packages/pyspark/python/